# LumenRoute Data Processing Assignment 1

Meeting Notes: 10/2/2025, Created by Jocelyn Wu (jocelyn.wu@berkeley.edu)

Data: Callsforservice_pst_15-19 - Calls for service from 2015 - 2019


In [19]:
# For analyzing basic data science 
import pandas as pd

## Task 1: Cleaning Data - Convert the Block Address into Latitude and Longitude

One of the other tasks is to convert an address into its repsective longitude and latitude.
To do that, there are 3 players you'll need to understand 
1) Geopy - a Python library that provides an easy way to access geocoding services like Google Maps, OpenStreetMap, Bing, etc (it's not automatically included like pandas in jupyterlab, so you'll need to instsall it once pip install geopy to gain access!)
2) Geocoder - A class (object) inside Geopy that talks to one specific geocoding service/map (you create one to communicate with a particular map)
3) Nomination - It's a specific geocoder service provided by OpenStreetMap (treat it like the actual "brain" that takes the address in, and return coordinates) 

In [43]:
!pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nomination(user_agent = "data_processing_team") #create a geolocator object


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


NameError: name 'Nomination' is not defined

In [ ]:
calls = pd.read_csv("CallForService.csv")   
# you might need to change the path if you're downloading the file by yourself!

In [45]:
calls.head(4)

,Unnamed: 0,Incident Number,CreateDatetime,Call Type,Source,Progress,Priority,Dispositions,Block Address,City,ZIP Code,Full Address
0,429618,2021-00060254,2021-12-30 23:58:00-08:00,CFS 23103 - Reckless Vehicle,911 Call,In Progress,Level 2,MDT Narrative Only,SAN PABLO AVE & CEDAR ST,BERKELEY,94702.0,"SAN PABLO AVE & CEDAR ST, BERKELEY CA 94702.0"
1,429617,2021-00060253,2021-12-30 23:40:47-08:00,CFS 1033A - Audible Alarm,Telephone,In Progress,Level 3,Building Secure and Tight,600 SAN LUIS RD,BERKELEY,94707.0,"600 SAN LUIS RD, BERKELEY CA 94707.0"
2,429616,2021-00060251,2021-12-30 23:23:06-08:00,CFS W911 - Wireless 911,911 Call,In Progress,Level 2,MDT Narrative Only,2600 10TH ST,BERKELEY,94710.0,"2600 10TH ST, BERKELEY CA 94710.0"
3,429615,2021-00060250,2021-12-30 23:00:55-08:00,CFS 1194 - Pedestrian Stop,Officer Initiated,In Progress,Level 0,MDT Narrative Only,1300 ASHBY AVE,BERKELEY,94702.0,"1300 ASHBY AVE, BERKELEY CA 94702.0"


In [50]:
#geocode needs a full address (including City and State) to recognize the address so that it can come up with the coordinates 
calls["ZIP Code"] = (calls["ZIP Code"].astype(str).str.replace(r"\.0$", "", regex=True).str.strip())
calls["Full Address"] = calls["Block Address"].astype(str) + ", " + calls["City"].astype(str) + " CA " + ", " + calls["ZIP Code"].astype(str)
calls.head(4)

,Unnamed: 0,Incident Number,CreateDatetime,Call Type,Source,Progress,Priority,Dispositions,Block Address,City,ZIP Code,Full Address
0,429618,2021-00060254,2021-12-30 23:58:00-08:00,CFS 23103 - Reckless Vehicle,911 Call,In Progress,Level 2,MDT Narrative Only,SAN PABLO AVE & CEDAR ST,BERKELEY,94702,"SAN PABLO AVE & CEDAR ST, BERKELEY CA , 94702"
1,429617,2021-00060253,2021-12-30 23:40:47-08:00,CFS 1033A - Audible Alarm,Telephone,In Progress,Level 3,Building Secure and Tight,600 SAN LUIS RD,BERKELEY,94707,"600 SAN LUIS RD, BERKELEY CA , 94707"
2,429616,2021-00060251,2021-12-30 23:23:06-08:00,CFS W911 - Wireless 911,911 Call,In Progress,Level 2,MDT Narrative Only,2600 10TH ST,BERKELEY,94710,"2600 10TH ST, BERKELEY CA , 94710"
3,429615,2021-00060250,2021-12-30 23:00:55-08:00,CFS 1194 - Pedestrian Stop,Officer Initiated,In Progress,Level 0,MDT Narrative Only,1300 ASHBY AVE,BERKELEY,94702,"1300 ASHBY AVE, BERKELEY CA , 94702"


In [53]:
#THIS CELL WILL TAKE A FEW MINUTES TO LOAD!
#Here's a function that uses the geocode. It will return the location's latitude and longitude in a tuple!
#Why a tuple? because Tuples represent a fixed pair of values that are immutable. However, lists represents a mutable collection. The point of a lat and long
# is to not be able to mutate, which is why we use tuples
def get_lat_long(address):   
    location = geolocator.geocode(address, timeout = 10)
    if location:  #if there is a location they could find, it will be "True"
        return (location.latitude, location.longitude)
    else:
        return (None, None)

coordinates = calls["Full Address"].apply(get_lat_long) #returns a Series 
calls["Latitude"], calls["Longitude"] = zip(*coordinates)  
#while zip() zips two seperate things together, zip(*) unzips things!

In [55]:
calls.head(4)

,Unnamed: 0,Incident Number,CreateDatetime,Call Type,Source,Progress,Priority,Dispositions,Block Address,City,ZIP Code,Full Address,Latitude,Longitude
0,429618,2021-00060254,2021-12-30 23:58:00-08:00,CFS 23103 - Reckless Vehicle,911 Call,In Progress,Level 2,MDT Narrative Only,SAN PABLO AVE & CEDAR ST,BERKELEY,94702,"SAN PABLO AVE & CEDAR ST, BERKELEY CA , 94702",37.875108,-122.293951
1,429617,2021-00060253,2021-12-30 23:40:47-08:00,CFS 1033A - Audible Alarm,Telephone,In Progress,Level 3,Building Secure and Tight,600 SAN LUIS RD,BERKELEY,94707,"600 SAN LUIS RD, BERKELEY CA , 94707",37.899662,-122.275461
2,429616,2021-00060251,2021-12-30 23:23:06-08:00,CFS W911 - Wireless 911,911 Call,In Progress,Level 2,MDT Narrative Only,2600 10TH ST,BERKELEY,94710,"2600 10TH ST, BERKELEY CA , 94710",37.858222,-122.289974
3,429615,2021-00060250,2021-12-30 23:00:55-08:00,CFS 1194 - Pedestrian Stop,Officer Initiated,In Progress,Level 0,MDT Narrative Only,1300 ASHBY AVE,BERKELEY,94702,"1300 ASHBY AVE, BERKELEY CA , 94702",37.852711,-122.283570


## Task 2: Cleaning Data - Create a New Column with only the Call Type (wihout the CFS in front) 

For this, I want to use pandas .str.split("- ") which outputs a list for each row! Note: .str is a vectorized strong accessor that let's you apply string operations (.split .replace etc) to every ELEMENT of a column

In [63]:
calls["Call Type Only"] = calls["Call Type"].str.split("- ").str[1]   #You first split it, and then you only keep the "second" part
calls.head(4)

,Unnamed: 0,Incident Number,CreateDatetime,Call Type,Source,Progress,Priority,Dispositions,Block Address,City,ZIP Code,Full Address,Latitude,Longitude,Call Type Only
0,429618,2021-00060254,2021-12-30 23:58:00-08:00,CFS 23103 - Reckless Vehicle,911 Call,In Progress,Level 2,MDT Narrative Only,SAN PABLO AVE & CEDAR ST,BERKELEY,94702,"SAN PABLO AVE & CEDAR ST, BERKELEY CA , 94702",37.875108,-122.293951,Reckless Vehicle
1,429617,2021-00060253,2021-12-30 23:40:47-08:00,CFS 1033A - Audible Alarm,Telephone,In Progress,Level 3,Building Secure and Tight,600 SAN LUIS RD,BERKELEY,94707,"600 SAN LUIS RD, BERKELEY CA , 94707",37.899662,-122.275461,Audible Alarm
2,429616,2021-00060251,2021-12-30 23:23:06-08:00,CFS W911 - Wireless 911,911 Call,In Progress,Level 2,MDT Narrative Only,2600 10TH ST,BERKELEY,94710,"2600 10TH ST, BERKELEY CA , 94710",37.858222,-122.289974,Wireless 911
3,429615,2021-00060250,2021-12-30 23:00:55-08:00,CFS 1194 - Pedestrian Stop,Officer Initiated,In Progress,Level 0,MDT Narrative Only,1300 ASHBY AVE,BERKELEY,94702,"1300 ASHBY AVE, BERKELEY CA , 94702",37.852711,-122.283570,Pedestrian Stop


## Task 3: One Hot Encode Columns for Priority

You can only One Hot Encode categorical data! Uses skit-Learn 

In [72]:
#first, you need to import OneHotEncoder from skitLearn
from sklearn.preprocessing import OneHotEncoder

In [80]:
calls["Priority"].unique()

array(['Level 2', 'Level 3', 'Level 0', 'Level 1', 'Level 4', 'Level'],
      dtype=object)

In [73]:
# Next, you have to "create" or "personalize" your One Hot Encorder
ohe = OneHotEncoder( handle_unknown = "ignore", sparse_output = False).set_output(transform = "pandas")
# bascially, it will ignore "unknown" or "weird" rows, and it will set output to become a pandas dataframe (instead of sparse output which is a sparse matrix) 

In [78]:
new_ohe = ohe.fit_transform(calls[["Priority"]])
new_ohe
# why do we need two [[]] instead of just []? 
# With [] it will just return a Series. But, we don't want that. Instead, we will [[]] so that it will return a dataframe


,Priority_Level,Priority_Level 0,Priority_Level 1,Priority_Level 2,Priority_Level 3,Priority_Level 4
0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
194,0.0,0.0,0.0,0.0,0.0,1.0
195,0.0,0.0,0.0,0.0,0.0,1.0
196,0.0,1.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.0,1.0,0.0,0.0


In [87]:
calls_merge = pd.merge( left = calls, right = new_ohe, left_index = True, right_index = True)
# if you want to merge based on indexes, there's a problem - there isn't a col name for indexes (left_on = "col name" or right_on = "col name") 
# instead, set left_index = True and right_index = True
calls_merge

,Unnamed: 0,Incident Number,CreateDatetime,Call Type,Source,Progress,Priority,Dispositions,Block Address,City,...,Full Address,Latitude,Longitude,Call Type Only,Priority_Level,Priority_Level 0,Priority_Level 1,Priority_Level 2,Priority_Level 3,Priority_Level 4
0,429618,2021-00060254,2021-12-30 23:58:00-08:00,CFS 23103 - Reckless Vehicle,911 Call,In Progress,Level 2,MDT Narrative Only,SAN PABLO AVE & CEDAR ST,BERKELEY,...,"SAN PABLO AVE & CEDAR ST, BERKELEY CA , 94702",37.875108,-122.293951,Reckless Vehicle,0.0,0.0,0.0,1.0,0.0,0.0
1,429617,2021-00060253,2021-12-30 23:40:47-08:00,CFS 1033A - Audible Alarm,Telephone,In Progress,Level 3,Building Secure and Tight,600 SAN LUIS RD,BERKELEY,...,"600 SAN LUIS RD, BERKELEY CA , 94707",37.899662,-122.275461,Audible Alarm,0.0,0.0,0.0,0.0,1.0,0.0
2,429616,2021-00060251,2021-12-30 23:23:06-08:00,CFS W911 - Wireless 911,911 Call,In Progress,Level 2,MDT Narrative Only,2600 10TH ST,BERKELEY,...,"2600 10TH ST, BERKELEY CA , 94710",37.858222,-122.289974,Wireless 911,0.0,0.0,0.0,1.0,0.0,0.0
3,429615,2021-00060250,2021-12-30 23:00:55-08:00,CFS 1194 - Pedestrian Stop,Officer Initiated,In Progress,Level 0,MDT Narrative Only,1300 ASHBY AVE,BERKELEY,...,"1300 ASHBY AVE, BERKELEY CA , 94702",37.852711,-122.283570,Pedestrian Stop,0.0,1.0,0.0,0.0,0.0,0.0
4,429614,2021-00060249,2021-12-30 23:00:43-08:00,CFS 1194 - Pedestrian Stop,Officer Initiated,In Progress,Level 0,MDT Narrative Only,1300 ASHBY AVE,BERKELEY,...,"1300 ASHBY AVE, BERKELEY CA , 94702",37.852711,-122.283570,Pedestrian Stop,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,429424,2021-00060063,2021-12-29 17:16:58-08:00,CFS SEC - Security Check,Officer Initiated,In Progress,Level 4,MDT Narrative Only,1900 4TH ST,BERKELEY,...,"1900 4TH ST, BERKELEY CA , 94710",37.868750,-122.300391,Security Check,0.0,0.0,0.0,0.0,0.0,1.0
195,429423,2021-00060062,2021-12-29 17:15:44-08:00,CFS SEC - Security Check,Officer Initiated,In Progress,Level 4,MDT Narrative Only,1000 GILMAN ST,BERKELEY,...,"1000 GILMAN ST, BERKELEY CA , 94710",37.879926,-122.297873,Security Check,0.0,0.0,0.0,0.0,0.0,1.0
196,429422,2021-00060059,2021-12-29 17:08:35-08:00,CFS T - Traffic Stop,Officer Initiated,In Progress,Level 0,"RIPA, MDT Narrative Only",BOWDITCH ST & HASTE ST,BERKELEY,...,"BOWDITCH ST & HASTE ST, BERKELEY CA , 94704",NaN,NaN,Traffic Stop,0.0,1.0,0.0,0.0,0.0,0.0
197,429421,2021-00060061,2021-12-29 17:06:37-08:00,CFS 10852 - Vehicle Damage,911 Call,Just Prior,Level 2,"RIPA, Primary Case (Case or Cold Theft), Homeless",2100 SHATTUCK AVE,BERKELEY,...,"2100 SHATTUCK AVE, BERKELEY CA , 94704",37.872026,-122.268117,Vehicle Damage,0.0,0.0,0.0,1.0,0.0,0.0


## Final Cleaned Dataset (Dropping Unneeded Columns)

In [92]:
cleaned_dataset = calls_merge.drop( ["Call Type" , "Priority_Level", "Priority", "Block Address", "City", "ZIP Code" ],axis= "columns") 
cleaned_dataset.head(4)

,Unnamed: 0,Incident Number,CreateDatetime,Source,Progress,Dispositions,Full Address,Latitude,Longitude,Call Type Only,Priority_Level 0,Priority_Level 1,Priority_Level 2,Priority_Level 3,Priority_Level 4
0,429618,2021-00060254,2021-12-30 23:58:00-08:00,911 Call,In Progress,MDT Narrative Only,"SAN PABLO AVE & CEDAR ST, BERKELEY CA , 94702",37.875108,-122.293951,Reckless Vehicle,0.0,0.0,1.0,0.0,0.0
1,429617,2021-00060253,2021-12-30 23:40:47-08:00,Telephone,In Progress,Building Secure and Tight,"600 SAN LUIS RD, BERKELEY CA , 94707",37.899662,-122.275461,Audible Alarm,0.0,0.0,0.0,1.0,0.0
2,429616,2021-00060251,2021-12-30 23:23:06-08:00,911 Call,In Progress,MDT Narrative Only,"2600 10TH ST, BERKELEY CA , 94710",37.858222,-122.289974,Wireless 911,0.0,0.0,1.0,0.0,0.0
3,429615,2021-00060250,2021-12-30 23:00:55-08:00,Officer Initiated,In Progress,MDT Narrative Only,"1300 ASHBY AVE, BERKELEY CA , 94702",37.852711,-122.283570,Pedestrian Stop,1.0,0.0,0.0,0.0,0.0


### Task 4: Create a function that will show summary of the crimes/statistics/etc 
### WIP Need Backend Insights on what Statistics they Want

In [97]:
# Count by Call Type 
count_by_call_type = cleaned_dataset.groupby("Call Type Only").size() # will return to me the # of calls per Call Type
count_by_call_type

Call Type Only
Abandoned Vehicle       8
Advice                  4
Aid to Citizen          2
Arson                   1
Ascertain 911           4
Audible Alarm          11
Auto Burglary           9
BMC Violation           1
Barking Dog             3
Battery                 4
Brandishing Weapon      1
Burglary                2
Civil Standby           1
Code 1 Assist           1
Dead Body               1
Disturbance            19
Domestic Violence       1
Exhibition of Speed     1
Family Disturbance      2
Flag Down               5
Forgery                 1
Found Property          2
Grand Theft             1
Hit & Run Prop.         4
LoJack Incident         1
Loud Report             1
Mentally Ill            4
Minor Injuries          4
Noise Disturbance       1
Outside Assist          4
PC Violation            1
Parking Violation       1
Pedestrian Stop         6
Prowler                 1
Reckless Vehicle        1
Recovered Stolen        1
Robbery                 4
Security Check         